# Implementation for the Trignometrically Exact Quadrature Features

This notebook will describe how to perform GP regression using Trignometrically exact quadrature featues. For convenience, we provide the quadrature weights and nodes for the quadrature rule that is exact for integrals of the form:
$$
\int_{-\pi}^{\pi} \exp(-(\gamma x)^2) f(\gamma x)
$$
When $f(x)$ is a cosine polynomial of low enough degree. These rules allows us to apply the fourier features method for the Square Expnential/Gaussian Kernel examine in the paper. Quadrature rules for other kernels that satisfy assumption one in the paper can be derived via the Golub-Welsh algorithm specified in the appendix. The pickle files in the folder "quad_weights" contain the weights for $\gamma = .85, 1, 1.15, 1.25$. 

We demonstrate how to run our method below. First we write a function to sample from a GP defined on 2 dimensions with RBF Kernel.

In [166]:
from model import *
import pandas as pd
from gpytorch.kernels import RBFKernel

#define function to generate data 
def gen_data(ls, ntrain = 10000):
    X = torch.rand((ntrain + 2000, 2)).cuda()

    what = RBFKernel().cuda()
    what._set_lengthscale(torch.tensor([[ls]]).cuda())
    covar = what(X, X) + DiagLinearOperator(torch.ones(X.shape[0]).cuda())*1e-3
    dist = MultivariateNormal(torch.zeros(X.shape[0]).cuda(), covar)
    Y = dist.sample()
    Y = Y + torch.randn(Y.shape).cuda()*.0225
    Xtrain, Ytrain = X[:ntrain], Y[:ntrain]
    Xtest, Ytest = X[ntrain:], Y[ntrain:]
    del covar
    del dist
    return 10*Xtrain, Ytrain, 10*Xtest, Ytest

Xtrain, Ytrain, Xtest, Ytest  = gen_data(.05, 20000)



We can get the nodes and the weights of the quadrature methods using the method below.

In [167]:
# get weights #
#methods include "trig"-trignometric quadrature features, "gl" - Gauss Legendre, "gh" - "gauss hermite", "rff" - random fourier features
# for quadrature methods $n$ indicates the degree of exactness in each dimension
# gamma supported for gamma = .85, 1, 1.15, 1.25
nodes, weights = nodes_weights(method = "trig", n = 12, d = 2, gamma = 1.15)
print(nodes.shape)
print(weights.shape)

torch.Size([288, 2])
torch.Size([288])


Define the and fit the GP:

In [168]:
gp = quadGP(nodes = nodes, weights = weights, num_c = 1, dtype = torch.float32, jitter = 1e-6)
gp.fit(Xtrain[:, None, :], Ytrain, iters = 1000, gl = False, rff = False,lr = .0025, update = 25)



Epoch: 100%|██████████| 1000/1000 [00:10<00:00, 99.28it/s, loss=-2.25] 


And make predictions

In [169]:
# get the predictive mean and variances of the GP
mu, var = gp.bpred(Xtest[:, None, :], Xtrain[:, None, :], Ytrain, gl = False, rff = False)
print("The Test NLL is {}".format(nll(Ytest, mu, var)))

The Test NLL is -2.35191011428833


We can do the same thing for gauss legendre features, except we have to set gl = True

In [170]:
nodes, weights = nodes_weights(method = "gl", n = 12, d = 2, gamma = 1.15)
gp = quadGP(nodes = nodes, weights = weights, num_c = 1, dtype = torch.float32, jitter = 1e-6)
gp.fit(Xtrain[:, None, :], Ytrain, iters = 1000, gl = True, rff = False,lr = .0025, update = 25)
mu, var = gp.bpred(Xtest[:, None, :], Xtrain[:, None, :], Ytrain, gl = True, rff = False)
print("The Test NLL is {}".format(nll(Ytest, mu, var)))

Epoch: 100%|██████████| 1000/1000 [00:09<00:00, 104.01it/s, loss=0.691]

The Test NLL is 0.6923612356185913
